In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import ollama

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('PERPLEXITY_API_KEY')
model_name = "sonar-pro"

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook


In [3]:
#openai = OpenAI()

#openai = OpenAI(base_url="https://api.perplexity.ai",api_key=api_key)
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "gemma3" # "sonar-pro"

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi! , which model are you ?"
response = openai.chat.completions.create(model=model_name, messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hi there! It's wonderful to hear from you – this is definitely my first conversation!

I'm Gemma, a large language model created by the Gemma team at Google DeepMind. I’m an open-weights model, which means I’m widely available for public use! 

It’s exciting to have you as my first user! 😊 

Do you have any questions for me, or would you like to chat about something specific?


In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [46]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [47]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [48]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [49]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [50]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model=model_name, messages=messages)
print(response.choices[0].message.content)

Seriously? You're asking *me* that? It's 4.  Don't embarrass me.


In [51]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [52]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [53]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = model_name,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [54]:
summarize("https://edwarddonner.com")

'Here\'s a markdown summary of the Edward Donner website:\n\n**Summary:**\n\nEdward Donner’s website showcases his expertise in artificial intelligence, particularly around Large Language Models (LLMs). He’s the co-founder and CTO of Nebula.io, an AI-powered platform for talent discovery and recruitment. Previously, he founded and led AI startup untapt, which was acquired in 2021 due to their patented matching model and award-winning platform. \n\n**Key Announcements/Content:**\n\n*   **Upcoming Courses:** There are several planned courses, including “Connecting my courses – become an LLM expert and leader” (May 28, 2025), a “2025 AI Executive Briefing" (May 18, 2025), and “The Complete Agentic AI Engineering Course” (April 21, 2025).\n*   **LLM Workshop:** A hands-on workshop focusing on LLM agents is also scheduled for January 23, 2025.'

In [55]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [56]:
display_summary("https://edwarddonner.com")

Here’s a short summary of the Edward Donner website, focusing on its content:

Edward Donner is a technology leader and entrepreneur focused on AI, particularly its application to talent acquisition. He’s the co-founder and CTO of Nebula.io, a company leveraging AI to help individuals discover their potential. Previously, he founded and led AI startup untapt, which was acquired in 2021 and boasts a patented matching model. 

He’s currently offering courses and workshops aimed at developing expertise in LLMs and Agentic AI Engineering. Announcements include a “Connecting my courses – become an LLM expert and leader” course planned for May 28, 2025, and a 2025 AI Executive Briefing scheduled for May 18, 2025. He also has a “The Complete Agentic AI Engineering Course” planned for April 21, 2025.

In [57]:
display_summary("https://cnn.com")

Okay, here's a breakdown of the content you've provided, categorized and analyzed for potential use in a system or for understanding the data's structure:

**1. Data Structure & Categories:**

*   **Top Level Categories:** (Broadest groupings)
    *   **News:** (General News, US Politics, World News – Africa, Americas, Asia, Australia, China, Europe, India, Middle East, UK)
    *   **Business & Finance:** (Markets, Investing)
    *   **Lifestyle & Culture:** (Health, Fitness, Food, Sleep, Mindfulness, Relationships, Entertainment, Style, Arts, Design, Luxury, Beauty)
    *   **Sports:** (Football, Tennis, Golf, Motorsports, US Sports, Olympics, Esports, Hockey)
    *   **Technology & Innovation:** (Tech, Innovate, Work Transformed, Innovative Cities)
    *   **Science & Exploration:** (Climate, Space, Unearthed)
*   **Sub-Categories:** Many of the top-level categories have further sub-categories (e.g., “Sports” has Football, Tennis, etc.). This shows a hierarchical structure.
*   **Features/Sections:** The data includes specific sections like “CNN Polls,” “CNN Heroes,” “CNN Fast,” etc., indicating dedicated features within CNN.

**2. Content Analysis (Example Snippets - Illustrative, not Exhaustive):**

*   **Politics:**
    *   “US Politics”: Mentions of “Trump” signals a key topic.
    *   “CNN Polls”:  Indicates a focus on public opinion.
*   **Business/Finance:**
    *   “Markets,” “Investing”: Categories clearly representing financial news.
*   **Lifestyle/Culture:**
    *   “Health,” “Fitness,” “Food,” “Sleep,” “Mindfulness” –  These categories reveal areas CNN covers related to well-being.
*   **Sports:** Very detailed breakdown of various sports.
*   **Science/Environment:** Focus on climate change (“Climate”) and space exploration (“Space,” “Unearthed”).
*   **Human Interest/Features:** Areas like “CNN Heroes” and “Inside Africa” suggest segments dedicated to celebrating positive stories.
*   **Technological Trends:** Sections like “Work Transformed” – highlights a focus on emerging trends.

**3. Potential Use Cases for this Data:**

*   **News Aggregation System:** This data could be used to build a system that aggregates news from CNN.
*   **Content Recommendation Engine:** Based on user preferences (e.g., "I like Sports"), the system could recommend relevant CNN articles.
*   **Topic Modeling & Trend Analysis:** Analyzing the frequency of words and categories could reveal trending topics and changes in CNN’s editorial focus.
*   **Content Tagging/Categorization:**  Automatically classify new CNN articles based on these categories.
*   **Data Training for NLP Models:**  This data could be used to train Natural Language Processing (NLP) models for tasks like sentiment analysis, topic extraction, or question answering.

**4.  Observations & Potential Improvements (For a System Building Perspective):**

*   **Granularity:** The data is quite granular. Do you need this level of detail for your use case?  Could you aggregate some categories for a higher-level view?
*   **Metadata:** This data *lacks* many crucial metadata fields.  For a robust system, you'd need to add fields like:
    *   **Publication Date/Time**
    *   **Author**
    *   **Article URL**
    *   **Sentiment Score** (Positive, Negative, Neutral)
    *   **Source:** (Specifically, “CNN”)
*   **Relationships:**  The data doesn’t explicitly show relationships *between* articles.  (e.g., "Article A discusses topic X, which is also covered in Article B").  Adding links between articles would create a more interconnected dataset.

**To help me give you an even more tailored response, could you tell me:**

*   What are you ultimately trying to *do* with this data? (e.g., build a news aggregator, create a recommendation system, train an NLP model?)
*   What are your key requirements? For example, is accuracy, speed, or cost the most important factors?